### 1. FindORFs.py

In [1]:
class CommandLine():
    """
    Handle the command line, usage and help requests.

    CommandLine uses argparse, now standard in 2.7 and beyond.
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.

    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.
    """
    def __init__(self, inOpts=None):
        """
         CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        :param inOpts:
        """
        import argparse
        self.parser = argparse.ArgumentParser(
            description='Finds the largest ORF in a DNA sequence',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
            )
        self.parser.add_argument('-lG', '--longestGene', action='store_true', help='longest Gene in an ORF')
        self.parser.add_argument('-mG', '--minGene', type=int, choices=(100, 200, 300, 500, 1000), action='store',
                                 help='minimum Gene length')
        self.parser.add_argument('-s', '--start', action='append', nargs='?',
                                 help='start Codon')  # allows multiple list options
        self.parser.add_argument('-v', '--version', action='version', version='%(prog)s 0.1')
        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)

class OrfFinder():
    """
    Takes in a sequence of a FASTA file and stores ORFs in a lists of lists.
    Attributes:
        attr1 (list): List of valid stop codons.
        attr2 (list): List of valid start codon.
        attr3 (dict): Dictionary of valid nucleotides.
    """
    stop_codons = ['TGA', 'TAG', 'TAA']
    start_codons = ['ATG']
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

    def __init__(self, seq):
        """Takes in sequence from users input file then creates a list of
        lists.
        Then creates ORFs list  which holds the frame, start, stop, length of the ORF.
        Args:
            param1 (str): String of DNA from FASTA file.
        """
        self.seq = seq
        self.orfs = []  # The lists where we save lists of ORFS and their position/length/frame.

    def findOrfs(self):
        """
        Find Orfs on top strand and return list of Orfs.
        Returns:
            (list): list of lists of ORFs found.
        """
        start_positions = []
        foundStart = 0
        foundCodon = 0

        for frame in range(0,3):  # We need to check for frames 1, 2, 3
            foundStart = 0  # Flag name for when finding codons and start codons.
            foundCodon = 0
            start_positions = []  # Clears the start position list for each frame.
            for i in range(frame, len(self.seq), 3):
                codon = self.seq[i:i+3]  # The codon is 3 nucleotides.
                if codon == 'ATG':  # When start codon is found.
                    start_positions.append(i)
                    foundStart = 1
                    foundCodon = 1

                if codon in OrfFinder.stop_codons and foundStart:
                    #print('ORF found')
                    start = start_positions[0] + 1 - frame
                    stop = i + 3
                    length = stop - start + 1
                    self.saveOrf((frame%3) + 1, start, stop, length)
                    start_positions = []
                    foundStart = 0
                    foundCodon = 1

                if not foundCodon and codon in OrfFinder.stop_codons:  # If no start codon was found but stop codon found.
                    #print("Dangling stop at 5' end")
                    start = 1
                    stop = i + 3
                    length = stop - start + 1
                    self.saveOrf((frame%3) + 1, start, stop, length)
                    start_positions = []
                    foundCodon = 1

            if foundStart:  # If no stop codon was found but start codon was found.
                #print("Dangling start at 3' end")
                start = start_positions[0] + 1
                stop = len(self.seq)
                length = stop - start + 1
                self.saveOrf((frame%3) + 1, start, stop, length)

        return self.orfs

    def findRevOrfs(self):
        """ Find Orfs on the bottom strand and returns that list of Orfs.
        Remember to fixup the orfList so that it refers to top strand coordinates and the rev frames.
        Returns:
            (list): list of lists of ORFs found on the reverse complement.
        """
        reverseComp = self.reverseComplement()
        start_positions = []
        foundStart = 0
        foundCodon = 0

        for frame in range(0, 3):  # We need to check for frames 1, 2, 3
            foundStart = 0  # Flag name for when finding codons and start codons.
            foundCodon = 0
            start_positions = []  # Clears the start position list for each frame.
            for i in range(frame, len(reverseComp), 3):
                codon = reverseComp[i:i + 3]  # The codon is 3 nucleotides.

                if codon == 'ATG':  # When start codon is found.
                    start_positions.append(i)
                    foundStart = 1
                    foundCodon = 1

                if codon in OrfFinder.stop_codons and foundStart:  # We have a start and stop codon.
                    stop = len(reverseComp) - start_positions[0]
                    start = len(reverseComp) - (i+2)
                    if frame == 1: stop += 1  # I'm not sure about this part.
                    elif frame == 2: stop += 2
                    length = stop - start + 1
                    self.saveOrf(-1 * ((frame%3) + 1), start, stop, length)
                    start_positions = []
                    foundStart = 0
                    foundCodon = 1

                if not foundCodon and codon in OrfFinder.stop_codons:  # If no start codon was found but stop codon found.
                    #print("Dangling stop at 5' end")
                    start = len(reverseComp) - i - 2
                    stop = len(reverseComp)
                    length = stop - start + 1
                    self.saveOrf(-1 * ((frame%3) + 1), start, stop, length)
                    start_positions = []
                    foundCodon = 1

            if foundStart:  # If no stop codon was found but start codon was found.
                #print("Dangling start at 3' end")
                start =  start_positions[0] + 1
                stop = 1
                length = stop - start + 1
                self.saveOrf(-1 * ((frame%3) + 1), start, stop, length)

        return self.orfs

    def saveOrf(self, frame, start, stop, length):
        """ Saves the information about the ORF that was found.
        This method is used in findOrfs and findRevOrfs.
        Args:
            param1 (int): Start position.
            param2 (int): Stop position.
            param3 (int): Length of my Orf. (Stop - Start)
            param4 (int): Which frame my Orf was found in.
        Returns:
            (list): A list of lists of Orfs with their appropriate information.
        """
        self.orfs.append([frame, start, stop, length])

    def reverseComplement(self):
        """ Helper method to take the reverse complement of a DNA seqeunce.
        Returns:
            (list): Reverse complement of DNA sequence.
        """
        return ''.join([self.complement[base] for base in self.seq[::-1]])  # Dictionary comprehension to find reverse complement of DNA sequence.

### 2. SequenceAnalysis.py

In [2]:
class NucParams:
    """Creates dictionaries to store fasta file information.

        Attributes:
            attr1 (dict): Dictionary of RNA codons.
            attr2 (dict): Dictionary of DNA codons.
            attr3 (dict): Dictionary of valid Nucleotides.
    """

    rnaCodonTable = {
                    # RNA codon table
    # U
        'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C',  # UxU
        'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C',  # UxC
        'UUA': 'L', 'UCA': 'S', 'UAA': '-', 'UGA': '-',  # UxA
        'UUG': 'L', 'UCG': 'S', 'UAG': '-', 'UGG': 'W',  # UxG
    # C
        'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R',  # CxU
        'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R',  # CxC
        'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R',  # CxA
        'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R',  # CxG
    # A
        'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S',  # AxU
        'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S',  # AxC
        'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R',  # AxA
        'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R',  # AxG
    # G
        'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G',  # GxU
        'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G',  # GxC
        'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G',  # GxA
        'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'   # GxG
    }

    dnaCodonTable = {key.replace('U','T'):value for key, value in rnaCodonTable.items()}

    validNucleotides = {'A': 0, 'C': 0, 'G': 0, 'T': 0, 'U': 0, 'N':0}

    def __init__(self):
        """
        Initializes dictionaries with appropriate keys, and values set to zero.
        """
        self.aminoAcidComposition = {}
        self.codonsComposition = {}
        self.nucleotideComposition = {}

        for aa in ProteinParam.aa2mw:  # Initializes Amino Acid dictionary.
            self.aminoAcidComposition[aa] = 0

        for codon in self.rnaCodonTable:  # Initializes Codon Composition dictionary.
            self.codonsComposition[codon] = 0

        for nuc in NucParams.validNucleotides:  # Initializes Nucleotide Composition dictionary.
            self.nucleotideComposition[nuc] = 0

    def addSequence(self, thisSequence):
        """
        This method accepts additional sequences and adds the data that you were given
        with the __init__ method.
        """
        for nuc in thisSequence:
            if nuc in NucParams.validNucleotides.keys():  # Checks if each nucleotide is valid.
                self.nucleotideComposition[nuc] += 1  # Counts how many nucleotides there are.

        rnaSequence = thisSequence.replace('T', 'U')  # Converts DNA sequence to RNA

        for start in range(0, len(rnaSequence), 3):
            codon = rnaSequence[start: start + 3]
            if codon in self.rnaCodonTable:
                self.codonsComposition[codon] += 1  # Adds RNA sequence to dictionary w/ count.
                aa = self.rnaCodonTable[codon]
                if aa != '-':
                    self.aminoAcidComposition[aa] += 1  # Adds amino acid to dictionary w/ count.


    def aaComposition(self):
        """
        This method will return a dictionary of counts over the 20 amino acids.
        """
        return self.aminoAcidComposition

    def nucComposition(self):
        """
        Returns the updated nucleotide composition dictionary from addSequence.
        """
        return self.nucleotideComposition

    def codonComposition(self):
        """
        This dictionary returns codons in RNA format and counts of codon.
        """
        return self.codonsComposition

    def nucCount(self):
        """
        This returns an integer value, summing every valid nucleotide found. This
        value should exactly equal the sum over the nucleotide composition dictionary.
        """
        nucTotal = 0
        for count in self.nucleotideComposition.values():
            nucTotal += count
        return nucTotal


class ProteinParam(str):
    """Creates a ProteinParam string object in upper case letters.

    Attributes:
        attr1 (dict): Dictionary of molecular weights of amino acids.
        attr2 (float): Molecular weight of H20.
        attr3 (dict): Dictionary of absorbance values at 280nm.
        attr4 (dict): Dictionary of positively charged amino acids.
        attr5 (dict): Dictionary of negatively charged amino acids.
        attr6 (float): Charge of the N terminus.
        attr7 (float): Charge of the C terminus.
        attr8 (dict): Dictionary of valid aa's.
    """

    aa2mw = {
        'A': 89.093, 'G': 75.067, 'M': 149.211, 'S': 105.093, 'C': 121.158,
        'H': 155.155, 'N': 132.118, 'T': 119.119, 'D': 133.103, 'I': 131.173,
        'P': 115.131, 'V': 117.146, 'E': 147.129, 'K': 146.188, 'Q': 146.145,
        'W': 204.225, 'F': 165.189, 'L': 131.173, 'R': 174.201, 'Y': 181.189
    }

    mwH2O = 18.015
    aa2abs280 = {'Y': 1490, 'W': 5500, 'C': 125}

    aa2chargePos = {'K': 10.5, 'R': 12.4, 'H': 6}
    aa2chargeNeg = {'D': 3.86, 'E': 4.25, 'C': 8.33, 'Y': 10}
    aaNterm = 9.69
    aaCterm = 2.34

    def __init__(self, protein):
        """Takes in sequence from users input then creates a list of
        strings and concatnates to create a string object.
        Then creates aaDictionary with the keys being the valid aa's of
        users input and the value of each one being their corresponding count.

        Args:
            param1 (str): String of amino acids.
        """
        myList = ''.join(protein).split()
        self.proteinString = ''.join(myList).upper()
        self.aaDictionary = {}

        for aa in self.aa2mw.keys():  # Iterates through aa2mw dictionary, the valid aa's.
            self.aaDictionary[aa] = float(self.proteinString.count(aa))  # Stores values.

    def aaCount(self):
        """ Iterates through every character in string and returns a count of valid
        amino acids. Ignores invalid characters.

        Returns:
            aaTotal (int): Total number of valid amino acids.
        """
        aaTotal = 0
        for aa in self.proteinString:
            if aa.upper() in self.aa2mw.keys():  # Checks if character in string is a valid aa.
                aaTotal += 1
        return aaTotal

    def pI(self):
        """Estimates the theoretical isoelectric point by finding the
        particular pH that yeilds a neutral net charge (as close to
        zero as possible, accurate to two decimal places).

        Returns:
            (float): Best pH.
        """
        bigCharge = 2 ** 11  # Choose a big number, this is our loop invariant.
        bestPH = 0
        particularPH = 0
        while particularPH < 14.01:
            charge = abs(self.__charge__(particularPH))  # 0 <= charge <= 14
            if charge < bigCharge:
                bigCharge = charge
                bestPH = particularPH
            particularPH += 0.01  # Needs to iterate through every pH in decimal places.
        return bestPH

    def aaComposition(self):
        """
        Returns:
            aaDictionary created in __init__ method.
        """
        return self.aaDictionary

    def __charge__(self, pH):
        """Calculates the net charge on the protein at specific pH using pKa of each
        charged amino acid, Nterminus and Cterminus.
        Args:
            param1 (float): pH value from pI method.
        Returns:
            (float): Net charge of the protein.
        """
        posCharge = 0
        for aa in self.aa2chargePos:  # Begin summation for posCharges.
            nAA = self.aaDictionary[aa]
            posCharge += nAA * ((10 ** self.aa2chargePos[aa])
                                / (10 ** self.aa2chargePos[aa] + 10 ** pH))
        posCharge += (10 ** self.aaNterm) / (10 ** self.aaNterm + 10 ** pH)  # Include the Nterm to posCharge.

        negCharge = 0
        for aa in self.aa2chargeNeg:
            nAA = self.aaDictionary[aa]
            negCharge += nAA * ((10 ** pH)
                                / (10 ** self.aa2chargeNeg[aa] + 10 ** pH))
        negCharge += (10 ** pH) / (10 ** self.aaCterm + 10 ** pH)

        netCharge = posCharge - negCharge

        return netCharge

    def molarExtinction(self):
        """Estimates the molar extinction coefficient based on the number
        of tyrosines, tryptophans, cysteines and their extinction
        coefficient at 280nm which can be found in dictionary(aa2abs280).

        Returns:
            (float): Molar extinction coefficient.
        """
        tyrosine = self.aaDictionary['Y'] * self.aa2abs280['Y']
        tryptophans = self.aaDictionary['W'] * self.aa2abs280['W']
        cysteines = self.aaDictionary['C'] * self.aa2abs280['C']
        molarEx = tyrosine + tryptophans + cysteines
        return molarEx

    def massExtinction(self):
        """Computes mass extinction by dividing molar extinction
        by the molecular weight of corresponding protein.

        Returns:
            (float): Mass extinction value.
        """
        myMW = self.molecularWeight()
        return self.molarExtinction() / myMW if myMW else 0.0

    def molecularWeight(self):
        """Calculates the molecular weight of the protein sequence by summing
        the weights of the individual Amino acids and excluding the waters
        that are released with peptide bond formation.

        Returns:
            (float): Molecular weight.
        """
        aaWeight = 0
        h2o = self.mwH2O * (self.aaCount() - 1)
        for aa, count in self.aaDictionary.items():
            aaWeight += (count * self.aa2mw[aa])  # Sums the weights of the individual aa's.
        return aaWeight - h2o  # Excludes the h2o's released with peptide bond formation.


import sys

class FastAreader:

    def __init__ (self, fname=''):
        '''contructor: saves attribute fname '''
        self.fname = fname

    def doOpen (self):
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta (self):

        header = ''
        sequence = ''

        with self.doOpen() as fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                line = fileH.readline()
            header = line[1:].rstrip()

            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()


        yield header,sequence

In [6]:
def main(myCommandLine=None):
    """Reads in a fasta file and outputs the ORFs frame, start, stop, and length position on a output file."""
    if myCommandLine is None:
        myCommandLine = CommandLine()
        if myCommandLine.args.longestGene:  # If the terminal sees lG flag variable start this part of code.
            fastaFile = FastAreader()
            for header, sequence in fastaFile.readFasta():
                print(header)
                orfData = OrfFinder(sequence)
                orfData.findOrfs()
                orfData.findRevOrfs()
                filteredList = filter(lambda orf: orf[3] > myCommandLine.args.minGene, orfData.orfs)  # Filters out the ORFS depending on the minGene arg.
                for frame, start, stop, length in sorted(filteredList, key=lambda orf: orf[3], reverse = True):  # Unzips filteredList and sorts the list by length.
                    print('{:+d} {:>5d}..{:>5d} {:>5d}'.format(frame, start, stop, length))
    else:
        myCommandLine = CommandLine(myCommandLine)


if __name__ == "__main__":
    main()